# Análisis de estadísticas de equipos de la MLB
En este notebook, analizaremos estadísticas básicas y avanzadas de equipos de la MLB para los últimos 5 años.

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

## Función para obtener datos de equipos
La siguiente función recupera estadísticas básicas de equipos de la MLB para un año específico usando la base de datos de Lahman.

In [12]:
# Función para cargar y filtrar datos
def load_and_filter_data(years):
    """
    Carga el archivo CSV de Lahman y selecciona las columnas relevantes.
    """
    teams_path = "../data/raw/Teams.csv"
    
    try:
        df = pd.read_csv(teams_path)
        # Filtramos por años
        df = df[df['yearID'].isin(years)]
        # Seleccionamos las columnas relevantes
        # cols = ['yearID', 'teamID', 'W', 'L', 'R', 'AB', 'H', 'BB', 'SO', 'HR', 'ER'] # basicas
        cols = ['yearID', 'teamID', 'W', 'L', 'R', 'AB', 'H', 'BB', 'SO',
               'HR', 'ER', '2B', '3B', 'HBP', 'SF', 'IPouts', 'HA', 'BBA',
               'SOA', 'E']
        return df[cols]
    except Exception as e:
        print(f"Error cargando los datos: {e}")
        return None

## Función para calcular métricas avanzadas
Calculamos métricas avanzadas como OBP, SLG, OPS y porcentaje de bases por bolas.

In [13]:
def calculate_metrics(df):
    """
    Calcula métricas de béisbol basadas en estadísticas básicas
    """
    # Calculamos OBP (On Base Percentage)
    df['OBP'] = (df['H'] + df['BB']) / (df['AB'] + df['BB'])
    
    # Calculamos SLG (Slugging Percentage)
    df['SLG'] = (df['H'] + df['HR'] * 3) / df['AB']
    
    # Calculamos OPS
    df['OPS'] = df['OBP'] + df['SLG']
    
    # Calculamos BB% (Base on Balls Percentage)
    df['BB%'] = df['BB'] / (df['AB'] + df['BB']) * 100
    
    return df

def calculate_advanced_metrics(df):
    """
    Calcula métricas avanzadas de béisbol basadas en estadísticas básicas
    """
    # Métricas Ofensivas
    # ------------------
    
    # Singles (1B) = Hits - (2B + 3B + HR)
    df['1B'] = df['H'] - (df['2B'] + df['3B'] + df['HR'])
    
    # Total Bases (TB)
    df['TB'] = df['1B'] + (2 * df['2B']) + (3 * df['3B']) + (4 * df['HR'])
    
    # Batting Average (AVG)
    df['AVG'] = df['H'] / df['AB']
    
    # On-Base Percentage (OBP)
    df['PA'] = df['AB'] + df['BB'] + df['HBP'] + df['SF']  # Plate Appearances
    df['OBP'] = (df['H'] + df['BB'] + df['HBP']) / df['PA']
    
    # Slugging Percentage (SLG)
    df['SLG'] = df['TB'] / df['AB']
    
    # On-base Plus Slugging (OPS)
    df['OPS'] = df['OBP'] + df['SLG']
    
    # Base on Balls Percentage (BB%)
    df['BB%'] = df['BB'] / df['PA'] * 100
    
    # Strike Out Percentage (K%)
    df['K%'] = df['SO'] / df['PA'] * 100
    
    # ISO (Isolated Power)
    df['ISO'] = df['SLG'] - df['AVG']
    
    # Métricas Defensivas y Pitcheo
    # -----------------------------
    
    # Innings Pitched (IP)
    df['IP'] = df['IPouts'] / 3
    
    # Earned Run Average (ERA)
    df['ERA'] = (df['ER'] * 9) / df['IP']
    
    # WHIP (Walks and Hits per Inning Pitched)
    df['WHIP'] = (df['HA'] + df['BBA']) / df['IP']
    
    # K/9 (Strikeouts per 9 innings)
    df['K/9'] = (df['SOA'] * 9) / df['IP']
    
    # BB/9 (Walks per 9 innings)
    df['BB/9'] = (df['BBA'] * 9) / df['IP']
    
    # K/BB Ratio
    df['K/BB'] = df['SOA'] / df['BBA']
    
    # Métricas de Equipo
    # -----------------
    
    # Winning Percentage
    df['WIN%'] = df['W'] / (df['W'] + df['L'])
    
    # Runs per Game
    df['R/G'] = df['R'] / (df['W'] + df['L'])
    
    # Defensive Efficiency Ratio (DER)
    # (1 - ((H - HR) + E) / (AB - K - HR + SF))
    df['DER'] = 1 - ((df['HA'] - df['HR'] + df['E']) / 
                     (df['AB'] - df['SO'] - df['HR'] + df['SF']))
    
    return df

## Recolectar y procesar datos
Analizamos los datos de los últimos 5 años, calculamos métricas avanzadas y combinamos los resultados.

## Mostrar resultados
Revisamos una muestra de los datos procesados, estadísticas descriptivas y valores faltantes.

In [14]:
# Años para los cuales queremos procesar los datos
years = range(2019, 2024)

# Cargar y procesar datos
data = load_and_filter_data(years)
if data is not None:
    # data = calculate_metrics(data)
    data = calculate_advanced_metrics(data)
    # Guardar el resultado
    data.to_csv("../data/processed/team_stats.csv", index=False)
    print("Datos procesados y guardados exitosamente.")

    # Mostrar muestra de los datos procesados
    print("Muestra de los datos:")
    display(data.head())

    # Estadísticas básicas
    print("\nEstadísticas descriptivas:")
    display(data.describe())
    
    # Verificar si existen valores faltantes
    print("\nValores faltantes por columna:")
    display(data.isnull().sum())
else:
    print("No se pudieron procesar los datos.")

Datos procesados y guardados exitosamente.
Muestra de los datos:


,yearID,teamID,W,L,R,AB,H,BB,SO,HR,...,ISO,IP,ERA,WHIP,K/9,BB/9,K/BB,WIN%,R/G,DER
2895,2019,BAL,54,108,729,5596,1379,462,1435.0,213,...,0.168156,1443.000000,5.594595,1.458766,7.783784,3.498960,2.224599,0.333333,4.500000,0.638896
2896,2019,BOS,84,78,901,5770,1554,590,1382.0,245,...,0.196534,1471.000000,4.698844,1.378654,9.991162,3.701564,2.699174,0.518519,5.561728,0.697636
2897,2019,CHA,72,89,708,5529,1443,378,1549.0,182,...,0.153011,1412.666667,4.899245,1.429920,8.358660,3.707881,2.254296,0.447205,4.397516,0.641514
2898,2019,CLE,93,69,769,5425,1354,563,1332.0,223,...,0.182673,1437.666667,3.762346,1.222815,9.440297,2.817065,3.351111,0.574074,4.746914,0.701736
2899,2019,DET,47,114,582,5549,1333,391,1595.0,149,...,0.147955,1433.000000,5.244243,1.459177,8.591766,3.366364,2.552239,0.291925,3.614907,0.605927



Estadísticas descriptivas:


,yearID,W,L,R,AB,H,BB,SO,HR,ER,...,ISO,IP,ERA,WHIP,K/9,BB/9,K/BB,WIN%,R/G,DER
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,...,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,2021.000000,70.773333,70.773333,647.133333,4770.033333,1176.506667,456.353333,1221.393333,174.046667,594.373333,...,0.167974,1251.131111,4.300330,1.308473,8.820461,3.318527,2.710747,0.499970,4.580981,0.682834
std,1.418951,24.317019,24.302939,200.407493,1408.778415,356.946366,140.128711,365.484599,61.028471,190.051337,...,0.022482,369.344783,0.618372,0.107607,0.721839,0.436199,0.464400,0.089327,0.522598,0.031930
min,2019.000000,19.000000,17.000000,219.000000,1752.000000,390.000000,147.000000,440.000000,51.000000,181.000000,...,0.115099,473.000000,2.796739,1.048002,6.720076,2.407244,1.873199,0.291925,3.438272,0.604020
25%,2020.000000,57.500000,59.500000,610.500000,5333.000000,1240.250000,421.500000,1218.750000,140.250000,556.500000,...,0.152046,1408.250000,3.858708,1.234356,8.398537,3.014291,2.310968,0.434568,4.234568,0.660969
50%,2021.000000,77.000000,75.500000,706.500000,5428.500000,1316.500000,500.000000,1373.500000,182.000000,635.500000,...,0.167937,1433.000000,4.244088,1.301546,8.848332,3.329937,2.658423,0.503086,4.533951,0.684904
75%,2022.000000,89.750000,88.000000,772.000000,5510.750000,1387.500000,556.750000,1453.000000,219.750000,728.500000,...,0.184614,1443.000000,4.789406,1.382108,9.324646,3.657170,3.064737,0.567593,4.905124,0.704255
max,2023.000000,111.000000,114.000000,947.000000,5770.000000,1554.000000,653.000000,1654.000000,307.000000,910.000000,...,0.226738,1474.333333,5.841655,1.601145,10.982143,4.399624,3.955414,0.716667,5.845679,0.765655



Valores faltantes por columna:


yearID    0
teamID    0
W         0
L         0
R         0
AB        0
H         0
BB        0
SO        0
HR        0
ER        0
2B        0
3B        0
HBP       0
SF        0
IPouts    0
HA        0
BBA       0
SOA       0
E         0
1B        0
TB        0
AVG       0
PA        0
OBP       0
SLG       0
OPS       0
BB%       0
K%        0
ISO       0
IP        0
ERA       0
WHIP      0
K/9       0
BB/9      0
K/BB      0
WIN%      0
R/G       0
DER       0
dtype: int64